In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [5]:
def reduce_mem(df):
    int_list = ['int', 'int32', 'int16']
    float_list = ['float', 'float32']
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in int_list:
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
        elif col_type in float_list:
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
    return df

In [4]:
train_df = pd.read_csv('./data/train_dataset.csv',sep='\t',names=['label','query_id','doc_id'] + ["feature_{}".format(i) for i in range(362)])
test_A_df = pd.read_csv('./data/test_dataset_A.csv',sep='\t',names=['query_id','doc_id'] + ["feature_{}".format(i) for i in range(362)])
test_B_df = pd.read_csv('./data/test_dataset_B.csv',sep='\t',names=['query_id','doc_id'] + ["feature_{}".format(i) for i in range(362)])

df = pd.concat([train_df, test_A_df, test_B_df], ignore_index=True)
df.shape

In [ ]:
feature_name = [i for i in df.columns if 'feature' in i]
target = 'label'
drop_col = []
for i in tqdm(feature_name):
    if df[i].std() == 0:
        drop_col.append(i)

df = df.drop(drop_col, axis=1)
for i in df.columns:
    df[i] = df[i].map(df[i].value_counts())

df = reduce_mem(df)

In [ ]:
with open('./input/origin_df.pkl', 'wb') as f:
    pickle.dump(df, f, protocol=4)